In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import eurostat

In [2]:
year = 2020
area_filter = ['TOTX4_MEMONIA','CRF1A1', 'CRF1A3', 'CRF1D1A', 'CRF2', 'CRF1A4', 'CRF1A2', 'CRF3', 'CRF5']
states_dict = {'Rakousko':'AT', 
               'Belgie':'BE', 
               'Bulharsko':'BG',
               'Kypr':'CY',
               'Česká Republika':'CZ',
               'Německo':'DE',
               'Dánsko':'DK',
               'Estonsko':'EE',
               'Španělsko':'ES',
               'Finsko':'FI',
               'Francie':'FR',
               'Chorvatsko':'HR',
               'Maďarsko':'HU',
               'Irsko':'IE',
               'Itálie':'IT',
               'Litva':'LT',
               'Lucembursko':'LU',
               'Lotyšsko':'LV',
               'Malta':'MT',
               'Nizozemsko':'NL',
               'Polsko':'PL',
               'Portugalsko':'PT',
               'Rumunsko':'RO',
               'Švédsko':'SE',
               'Slovinsko':'SI',
               'Slovensko':'SK',}

Potřebuješ přidat ty sektory, ze kterých pak tvoříš ty obdelníčky 
- celkové emise: TOTX4_MEMONIA
- Energetika: CRF1A1
- Doprava: [CRF1A3 + CRF1D1A]
- Průmyslové procesy: CRF2
- Domácnosti a instituce: CRF1A4
- Spalování v průmyslu: CRF1A2
- Zemědělství: CRF3
- Odpadové hospodářství: CRF5
- Jiné: rozdíl do celku


In [3]:
# tohle už dělá to co od toho potřebuju :)
def _get_basic_emis_data(year, states_dict, area_filter):
    df_sorce_emis = eurostat.get_data_df('env_air_gge')
    df_basic_emis = df_sorce_emis.rename(columns={'geo\\time': 'geo'})
    df_basic_emis = df_basic_emis.query("src_crf == @area_filter and unit == 'MIO_T' and airpol == 'GHG'") # používám tady ten area_filter dobře?
    df_basic_emis = df_basic_emis.loc[df_basic_emis['geo'].isin(states_dict.values())] # add states according to states_dict
    df_basic_emis = df_basic_emis[["src_crf","geo", year]]
    return df_basic_emis

In [9]:
df_basic_emis = _get_basic_emis_data(year, states_dict, area_filter)

In [10]:
df_basic_emis.head()

,src_crf,geo,2020
13369,CRF1A1,AT,8.80730
13370,CRF1A1,BE,19.01476
13371,CRF1A1,BG,18.24960
13373,CRF1A1,CY,3.03325
13374,CRF1A1,CZ,41.60333


In [11]:
df_basic_emis.query("geo == 'AT'")

,src_crf,geo,2020
13369,CRF1A1,AT,8.80730
13513,CRF1A2,AT,10.54950
13801,CRF1A3,AT,21.18303
14193,CRF1A4,AT,9.02680
14624,CRF1D1A,AT,1.05298
14765,CRF2,AT,15.48929
16259,CRF3,AT,6.96425
18408,CRF5,AT,1.20924
19196,TOTX4_MEMONIA,AT,74.64500


POSTUP
- spoj řádky [CRF1A3 + CRF1D1A] pro kolonku doprava
- přidat kolonku "jině" = dopočíst rozdíl do celku

ODTUD zatím jen tupě přejaté

In [6]:
# from EU_emissions_ntb
# get demography data
def _get_demo_data(year, states_dict):
    """Import data from Eurostat,
    rename columns, select certain data.
    """
    df_sorce_demo = eurostat.get_data_df('demo_pjan')
    df_demo = df_sorce_demo.rename(columns={'geo\\time': 'geo'})
    df_demo = df_demo.query("age == 'TOTAL' and sex == 'T'")
    df_demo = df_demo.loc[df_demo['geo'].isin(states_dict.values())] # add states according to states_dict
    df_demo = df_demo[["geo", year]]
    return df_demo

In [7]:
# from EU_emissions_ntb
# merge dataframes
def _merge_df(year, df_emis, df_demo, states_dict):
    """merge dataframes,
    adjust dataframe.
    """
    # merge df
    df_merged = df_emis.merge(df_demo, on='geo', suffixes=('_emis', '_popul')) # "suffixes" rename columns 
    
    # adjust df
    df_merged['em_per_capita'] = (df_merged[f'{year}_emis'] * 1000000) / df_merged[f'{year}_popul']
    df_merged['country'] = states_dict.keys()
    df_merged = df_merged.set_index('country')
    df_merged = df_merged.sort_values(f'{year}_emis', ascending = False)
    return df_merged

In [8]:
df_emis = _get_emis_data(year, states_dict) # tohle už tu ted není - nahrad "df_basic_emis"
df_demo = _get_demo_data(year, states_dict)
df_merged = _merge_df(year, df_emis, df_demo, states_dict)

df_merged

NameError: name '_get_emis_data' is not defined

Jiné možná k něčemu použitelné

In [ ]:
df_basic_emis['src_crf'].unique()